# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_human_or_worm"
TOKENIZER = "kmer"
KMER = 2


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_human_or_worm 0 kmer 2


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 22
{'NG': 20, 'AN': 19, 'NT': 18, 'GC': 14, 'TC': 13, 'NN': 17, 'GG': 12, 'GA': 11, 'TG': 9, 'AG': 2, 'AA': 8, 'TA': 6, '<pad>': 21, 'CT': 7, 'CG': 16, 'CC': 5, 'GT': 15, 'AT': 1, 'AC': 10, 'CA': 4, 'TT': 3, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  100


not all sequences are of the same length


In [9]:
tokenizer(train_dset[1][0])

['TT',
 'AA',
 'AC',
 'TT',
 'TA',
 'AA',
 'TT',
 'TT',
 'CC',
 'CA',
 'AA',
 'CT',
 'AC',
 'AG',
 'GA',
 'AC',
 'CC',
 'TT',
 'TG',
 'TT',
 'GA',
 'AT',
 'TT',
 'TA',
 'AC',
 'AG',
 'AT',
 'TT',
 'TT',
 'GA',
 'GC',
 'AC',
 'GA',
 'AT',
 'AC',
 'TA',
 'CA',
 'TT',
 'TT',
 'AA',
 'GA',
 'CA',
 'AT',
 'TT',
 'CC',
 'TC',
 'AT',
 'TC',
 'AT',
 'GT',
 'TA',
 'CT',
 'TC',
 'TT',
 'AT',
 'TT',
 'AA',
 'AG',
 'AA',
 'AT',
 'TT',
 'AT',
 'CA',
 'GA',
 'AA',
 'TT',
 'TA',
 'TT',
 'TA',
 'TT',
 'GA',
 'GG',
 'AC',
 'TA',
 'TG',
 'TA',
 'AT',
 'AA',
 'AC',
 'AT',
 'TT',
 'TT',
 'TA',
 'AT',
 'CA',
 'AC',
 'AT',
 'CT',
 'TT',
 'TT',
 'AT',
 'GA',
 'AC',
 'AA',
 'TT',
 'GA',
 'CC',
 'AT',
 'AG',
 'TG']

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 88.7%, Avg loss: 0.556687 

Epoch 1


Train metrics: 
 Accuracy: 91.1%, Avg loss: 0.543690 

Epoch 2


Train metrics: 
 Accuracy: 90.5%, Avg loss: 0.543960 

Epoch 3


Train metrics: 
 Accuracy: 92.3%, Avg loss: 0.539384 

Epoch 4


Train metrics: 
 Accuracy: 92.5%, Avg loss: 0.539021 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  11385.917334794998 ; fp  923.6455709185684
recall  0.9108733867835999 ; precision  0.9249652016084298
num_batches 782
correct 22980
size 25000
Test metrics: 
 Accuracy: 0.919200, F1 score: 0.917865, Avg loss: 0.542362 



(0.9192, 0.9178652101261209)